# Polar Express Advanced Demo

This notebook demonstrates the capabilities of the Polar Express library, which allows you to convert JSONPath expressions to Polars expressions and build data transformation pipelines.

## What is Polar Express?

Polar Express is a library that bridges the gap between JSONPath expressions and the Polars DataFrame library. It allows you to:

1. Extract data from nested JSON using JSONPath expressions
2. Convert JSONPath to native Polars expressions
3. Build complex data transformation pipelines
4. Handle nested arrays, filtering, and complex selection criteria

In [ ]:
import json
import polars as pl
from polar_express import JSONPathExpr, PolarMapper
from polar_express.core.table_mapper import TableMapper

## Creating Complex Nested JSON Data

Let's create a complex nested JSON dataset to demonstrate the capabilities of Polar Express. This dataset models a book catalog system containing:

- Multiple levels of nesting
- Arrays of objects at different levels
- Properties that can be filtered with predicates
- Various data types (strings, numbers, booleans, null)

The structure will allow for testing complex JSONPath expressions like: `$.stores[*].sections.shelves[*].books[?(@.format == "hardcover")].title`

In [2]:
# Generate complex nested JSON data for a book inventory system
book_inventory = {
    "metadata": {"version": "2.3", "generated": "2025-04-05", "source": "Bookworm Database"},
    "stores": [
        {
            "id": "store001",
            "name": "Downtown Books",
            "open": True,
            "sections": {
                "id": "section101",
                "description": "Fiction Section",
                "shelves": [
                    {
                        "id": "shelf201",
                        "capacity": 42,
                        "genres": ["fantasy", "sci-fi"],
                        "books": [
                            {"format": "hardcover", "title": "The Midnight Library", "rating": 4.2},
                            {"format": "paperback", "title": "Project Hail Mary", "rating": 4.8},
                            {"format": "hardcover", "title": "Dune", "rating": 4.7},
                        ],
                    },
                    {
                        "id": "shelf202",
                        "capacity": 35,
                        "genres": ["mystery"],
                        "books": [
                            {"format": "ebook", "title": "The Silent Patient", "rating": 4.5},
                            {"format": "hardcover", "title": "The Thursday Murder Club", "rating": 4.3},
                        ],
                    },
                ],
            },
        },
        {
            "id": "store002",
            "name": "Westside Readers",
            "open": False,
            "sections": {
                "id": "section102",
                "description": "Non-Fiction Section",
                "shelves": [
                    {
                        "id": "shelf203",
                        "capacity": 28,
                        "genres": ["biography", "history"],
                        "books": [
                            {"format": "hardcover", "title": "A Promised Land", "rating": 4.9},
                            {"format": "paperback", "title": "Sapiens", "rating": 4.6},
                        ],
                    }
                ],
            },
        },
        {
            "id": "store003",
            "name": "Harbor Bookshop",
            "open": True,
            "sections": {
                "id": "section103",
                "description": "Mixed Collection",
                "shelves": [
                    {
                        "id": "shelf204",
                        "capacity": 50,
                        "genres": ["self-help"],
                        "books": [
                            {"format": "hardcover", "title": "Atomic Habits", "rating": 4.8},
                            {"format": "hardcover", "title": "Deep Work", "rating": 4.7},
                        ],
                    },
                    {
                        "id": "shelf205",
                        "capacity": 32,
                        "genres": ["cooking", "travel"],
                        "books": [
                            {"format": "paperback", "title": "Salt Fat Acid Heat", "rating": 4.8},
                            {"format": "ebook", "title": "World Travel", "rating": 4.4},
                            {"format": "hardcover", "title": "The Food Lab", "rating": 4.9},
                        ],
                    },
                ],
            },
        },
    ],
}

# Print data structure explanation
print("Book Inventory Structure:")
print("- stores[]: Array of 3 bookstore objects, each with an id, name, open status, and sections")
print("  - sections: Object with id, description, and an array of shelves")
print("    - shelves[]: Array of shelf objects, each with id, capacity, genres array, and a books array")
print("      - books[]: Array of book objects, each with format, title, and rating")
print("\nThis structure allows testing complex JSONPath expressions like:")
print('$.stores[*].sections.shelves[*].books[?(@.format == "hardcover")].title')

# Create a Polars DataFrame with the JSON data
df = pl.DataFrame({"document_id": ["catalog1"], "data": [json.dumps(book_inventory)]})
df

Book Inventory Structure:
- stores[]: Array of 3 bookstore objects, each with an id, name, open status, and sections
  - sections: Object with id, description, and an array of shelves
    - shelves[]: Array of shelf objects, each with id, capacity, genres array, and a books array
      - books[]: Array of book objects, each with format, title, and rating

This structure allows testing complex JSONPath expressions like:
$.stores[*].sections.shelves[*].books[?(@.format == "hardcover")].title


document_id,data
str,str
"""catalog1""","""{""metadata"": {""version"": ""2.3""…"


## JSONPath Expression Examples

Let's define some interesting JSONPath expressions to extract data from our book inventory structure:

In [3]:
# Example JSONPath expressions for our book inventory data
jsonpath_examples = [
    # Basic properties
    ("$.data.metadata.version", "Extract the metadata version"),
    # All store names
    ("$.data.stores[*].name", "Extract all bookstore names"),
    # Accessing nested object properties
    ("$.data.stores[*].sections.description", "Extract all section descriptions"),
    # Multiple array traversal
    ("$.data.stores[*].sections.shelves[*].capacity", "Extract all shelf capacities"),
    # Triple nested array with filtering
    (
        '$.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover")].title',
        "Extract titles of all hardcover books",
    ),
    # Filtering with logical operators
    ("$.data.stores[?(@.open == true)].name", "Extract names of open bookstores"),
    # Complex filtering with number comparison
    ("$.data.stores[*].sections.shelves[?(@.capacity > 40)].id", "Extract ids of shelves with capacity > 40"),
    # Deeply nested filtering with multiple conditions
    (
        '$.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover" && @.rating > 4.5)].title',
        "Extract titles of hardcover books with rating > 4.5",
    ),
]

# Print the examples
for i, (path, description) in enumerate(jsonpath_examples, 1):
    print(f"{i}. {path}\n   {description}\n")

1. $.data.metadata.version
   Extract the metadata version

2. $.data.stores[*].name
   Extract all bookstore names

3. $.data.stores[*].sections.description
   Extract all section descriptions

4. $.data.stores[*].sections.shelves[*].capacity
   Extract all shelf capacities

5. $.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover")].title
   Extract titles of all hardcover books

6. $.data.stores[?(@.open == true)].name
   Extract names of open bookstores

7. $.data.stores[*].sections.shelves[?(@.capacity > 40)].id
   Extract ids of shelves with capacity > 40

8. $.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover" && @.rating > 4.5)].title
   Extract titles of hardcover books with rating > 4.5



## Testing Individual JSONPath Expressions

Let's test each of the JSONPath expressions we defined above:

In [4]:
# Test each JSONPath expression individually
for path, description in jsonpath_examples:
    print(f"Expression: {path}")
    print(f"Description: {description}")

    # Create JSONPathExpr and extract data
    expr = JSONPathExpr(path, alias=description)
    result = df.with_columns(expr.expr)

    # Show the result
    display(result.select(expr.alias))
    print("-" * 80)

Expression: $.data.metadata.version
Description: Extract the metadata version


Extract the metadata version
str
"""2.3"""


--------------------------------------------------------------------------------
Expression: $.data.stores[*].name
Description: Extract all bookstore names


Extract all bookstore names
list[str]
"[""Downtown Books"", ""Westside Readers"", ""Harbor Bookshop""]"


--------------------------------------------------------------------------------
Expression: $.data.stores[*].sections.description
Description: Extract all section descriptions


...

--------------------------------------------------------------------------------
Expression: $.data.stores[*].sections.shelves[?(@.capacity > 40)].id
Description: Extract ids of shelves with capacity > 40


Extract ids of shelves with capacity > 40
str
"""[{""id"":""store001"",""name"":""Down…"


--------------------------------------------------------------------------------
Expression: $.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover" && @.rating > 4.5)].title
Description: Extract titles of hardcover books with rating > 4.5


Extract titles of hardcover books with rating > 4.5
str
"""[{""id"":""store001"",""name"":""Down…"


--------------------------------------------------------------------------------


In [10]:
path = '$.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover")].title'
expr = JSONPathExpr(path)
str(expr.expr)

'col("data").str.json_path_match([String($.stores)]).strict_cast(String)'

## Building a Data Transformation Pipeline

Now, let's build a more complex data transformation pipeline using the PolarMapper class:

In [6]:
# Create a PolarMapper to build an ETL pipeline
mapper = PolarMapper()

# Step 1: Extract all bookstore names
store_names = JSONPathExpr("$.data.stores[*].name", alias="store_names")
mapper.add_with_columns_step([store_names.expr])

# Step 2: Extract titles of all hardcover books
hardcover_titles = JSONPathExpr(
    '$.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover")].title', alias="hardcover_titles"
)
mapper.add_with_columns_step([hardcover_titles.expr])

# Step 3: Extract high-rated hardcover books (rating > 4.5)
high_rated_hardcovers = JSONPathExpr(
    '$.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover" && @.rating > 4.5)].title',
    alias="high_rated_hardcovers",
)
mapper.add_with_columns_step([high_rated_hardcovers.expr])

# Step 4: Extract shelf IDs with high capacity
high_capacity_shelves = JSONPathExpr(
    "$.data.stores[*].sections.shelves[?(@.capacity > 40)].id", alias="high_capacity_shelves"
)
mapper.add_with_columns_step([high_capacity_shelves.expr])

# Step 5: Select only the columns we want
mapper.add_select_step(
    [
        pl.col("document_id"),
        pl.col("store_names"),
        pl.col("hardcover_titles"),
        pl.col("high_rated_hardcovers"),
        pl.col("high_capacity_shelves"),
    ]
)

# Apply the transformation pipeline
result = mapper.map(df)

print("ETL Pipeline Results:")
print(result)

ETL Pipeline Results:
shape: (1, 5)
┌─────────────┬─────────────────────┬────────────────────┬────────────────────┬────────────────────┐
│ document_id ┆ store_names         ┆ hardcover_titles   ┆ high_rated_hardcov ┆ high_capacity_shel │
│ ---         ┆ ---                 ┆ ---                ┆ ers                ┆ ves                │
│ str         ┆ list[str]           ┆ str                ┆ ---                ┆ ---                │
│             ┆                     ┆                    ┆ str                ┆ str                │
╞═════════════╪═════════════════════╪════════════════════╪════════════════════╪════════════════════╡
│ catalog1    ┆ ["Downtown Books",  ┆ [{"id":"store001", ┆ [{"id":"store001", ┆ [{"id":"store001", │
│             ┆ "Westside R…        ┆ "name":"Down…      ┆ "name":"Down…      ┆ "name":"Down…      │
└─────────────┴─────────────────────┴────────────────────┴────────────────────┴────────────────────┘


## Examining the Mapper

Let's examine the mapper we created to understand the transformation pipeline:

In [7]:
print("Mapper description:")
print(mapper.describe())

# Save the mapper to JSON for later use
mapper_json = mapper.to_json()
print("\nMapper JSON representation:")
print(mapper_json)

Mapper description:
This mapper performs the following operations on the input DataFrame:

Step 1: Add columns .when([([(col("data").str.json_path_match([String($.stores)])) == (String([]))]) | (col("data").str.json_path_match([String($.stores)]).is_null())]).then(null).otherwise(col("data").str.json_path_match([String($.stores)]).str.json_decode().eval()).alias("store_names")

Step 2: Add columns col("data").str.json_path_match([String($.stores)]).strict_cast(String).alias("hardcover_titles")

Step 3: Add columns col("data").str.json_path_match([String($.stores)]).strict_cast(String).alias("high_rated_hardcovers")

Step 4: Add columns col("data").str.json_path_match([String($.stores)]).strict_cast(String).alias("high_capacity_shelves")

Step 5: Select columns col("document_id"), col("store_names"), col("hardcover_titles"), col("high_rated_hardcovers"), col("high_capacity_shelves")



Mapper JSON representation:
{
  "steps": [
    {
      "type": "with_columns",
      "exprs": [
      

## Loading a Mapper from JSON

You can save the mapper to JSON and load it back to reuse the same transformation:

In [8]:
# Create a new mapper from the JSON
new_mapper = PolarMapper.from_json(mapper_json)

# Apply the loaded mapper to the same data
new_result = new_mapper.map(df)

print("Results using loaded mapper:")
print(new_result)

# Verify the results are the same
print(f"\nResults match: {new_result.equals(result)}")

AttributeError: type object 'PolarMapper' has no attribute 'from_json'

## Using TableMapper for Schema Definition

The TableMapper class provides a convenient way to define table schemas using JSONPath expressions. Let's see how we can use it with our book inventory data:

In [ ]:
# Create a TableMapper to define a schema for the book inventory
book_table = TableMapper()

# Add columns for basic store information
book_table.add_column("store_id", "$.data.stores[*].id")
book_table.add_column("store_name", "$.data.stores[*].name")
book_table.add_column("is_open", "$.data.stores[*].open")

# Add columns for section information
book_table.add_column("section_id", "$.data.stores[*].sections.id")
book_table.add_column("section_desc", "$.data.stores[*].sections.description")

# Print the defined schema
print("Book inventory schema:")
print(book_table)

# Apply the table schema to our DataFrame
store_info = book_table.map(df)
print("\nStore information extracted using TableMapper:")
print(store_info)

In [ ]:
# Creating a TableMapper for book details using method chaining
book_details = (
    TableMapper()
    .add_column("store_name", "$.data.stores[*].name")
    .add_column("book_title", "$.data.stores[*].sections.shelves[*].books[*].title")
    .add_column("book_format", "$.data.stores[*].sections.shelves[*].books[*].format")
    .add_column("book_rating", "$.data.stores[*].sections.shelves[*].books[*].rating")
)

# Print the schema
print("Book details schema:")
print(book_details)

# Apply the mapper to extract book details
book_result = book_details.map(df)
print("\nBook details extracted using TableMapper:")
print(book_result)

In [ ]:
# Using a dictionary to define a schema
hardcover_schema = {
    "store_name": "$.data.stores[*].name",
    "shelf_id": "$.data.stores[*].sections.shelves[*].id",
    "hardcover_title": '$.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover")].title',
    "hardcover_rating": '$.data.stores[*].sections.shelves[*].books[?(@.format == "hardcover")].rating'
}

# Create a TableMapper from a schema dictionary
hardcover_mapper = TableMapper.from_schema_dict(hardcover_schema)

# Print the schema
print("Hardcover books schema:")
print(hardcover_mapper)

# Apply the mapper to extract hardcover book details
hardcover_result = hardcover_mapper.map(df)
print("\nHardcover books extracted using TableMapper:")
print(hardcover_result)

In [ ]:
# Creating high-rated books filter
high_rating_schema = {
    "store_name": "$.data.stores[*].name",
    "high_rated_book": "$.data.stores[*].sections.shelves[*].books[?(@.rating > 4.5)].title",
    "rating": "$.data.stores[*].sections.shelves[*].books[?(@.rating > 4.5)].rating"
}

# Create and apply the mapper
high_rated_mapper = TableMapper().from_dict(high_rating_schema)
high_rated_result = high_rated_mapper.map(df)

print("High-rated books (rating > 4.5):")
print(high_rated_result)

# Get the schema as a dictionary
print("\nSchema as dictionary:")
print(high_rated_mapper.get_schema())

## Converting TableMapper to PolarMapper

A TableMapper can be converted to a PolarMapper for more complex transformations:

In [ ]:
# Convert TableMapper to PolarMapper for additional transformations
store_mapper = TableMapper()
store_mapper.add_column("store_id", "$.data.stores[*].id")
store_mapper.add_column("store_name", "$.data.stores[*].name")
store_mapper.add_column("is_open", "$.data.stores[*].open")

# Convert to PolarMapper
polar_mapper = store_mapper.to_mapper()

# Add additional transformation steps
polar_mapper.add_with_columns_step([
    pl.col("is_open").alias("status").map_elements(lambda x: "Open" if x else "Closed")
])

# Add a filter step to only show open stores
polar_mapper.add_filter_step(pl.col("is_open") == True)

# Apply the enhanced mapper
enhanced_result = polar_mapper.map(df)

print("Enhanced mapping with additional transformations:")
print(enhanced_result)

## Conclusion

The Polar Express library provides multiple ways to work with nested JSON data:

1. **Individual JSONPath expressions** - For simple extraction of specific fields
2. **PolarMapper** - For building complex transformation pipelines with multiple steps
3. **TableMapper** - For defining table schemas using JSONPath expressions

Benefits of using TableMapper:
- Declarative schema definition using JSONPath
- Automatic conversion to Polars expressions
- Support for nested data, arrays, and filtering with predicates
- Easy conversion to PolarMapper for more complex transformations
- Serialization and deserialization of schemas

This makes Polar Express a powerful tool for data engineers and analysts working with complex JSON data in polars DataFrames.